<a href="https://colab.research.google.com/github/dragonsan17/crime_det/blob/master/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import csv
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [11]:
from google.colab import drive 
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
#my data
data_path_1 = '/content/gdrive/My Drive/crime_det/final_tweet_data_25k.csv'

with open(data_path_1, newline='') as f_1:
    reader = csv.reader(f_1)
    train_data = list(reader)
#sourav's data
data_path_2 = '/content/gdrive/My Drive/crime_det/final_tweet_data_2.csv'

with open(data_path_2, newline='') as f_2:
    reader = csv.reader(f_2)
    valid_data = list(reader)
print(len(train_data))
print(len(valid_data))

464
415


In [0]:
# crimes = ['hostage','murder','robbery','con','fraud','kidnap','rape','terrorism','terror','crime','protest','pelt','arson','clash']
# # crimes = ['murder']
# crime_index = {}
# for i in range(len(crimes)):
#   crime_index[crimes[i]] = i

# # crime_index['theft'] = crime_index['robbery']

In [0]:
# new_train_data = []
# new_valid_data = []
# # for i in range(len(train_data)):
# #   if(train_data[i][0]!='murder'):
# #     continue
# #   else:
# #     new_train_data.append(train_data[i])
# for i in range(len(valid_data)):
#   if(valid_data[i][1]!='-1'):
#     continue
#   else:
#     new_valid_data.append(valid_data[i])
# # train_data = new_train_data
# valid_data = new_valid_data

# # print(len(valid_data))

In [0]:
def data_prep(data):
  token_list = []
  label_list = []

  for crime,num,s_token,tweet in data:
    token_list.append(s_token[1:-1].split(', '))
    label_list.append(max(int(num),0))

  for i in range(len(token_list)):
    for j in range(len(token_list[i])):
      token_list[i][j] = token_list[i][j][1:-1]
  # print(len(token_list))
  tokens = {}
  crime_info = {}
  # print(len(data))
  for i in range(len(data)):
    tokens[data[i][3]] = token_list[i]
    crime_info[data[i][3]] = []
  for i in range(len(data)):
    crime_info[data[i][3]].append(max(int(data[i][1]),0))

  x = []
  y = []

  for tweet in tokens:
    x.append(tokens[tweet])
    label = 0
    for num in crime_info[tweet]:
      label = max(num,0)
    y.append(label)
  return x,y 

x_train,y_train = data_prep(train_data)
x_valid,y_valid = data_prep(valid_data)

training_labels_final = np.array(y_train).astype(int)
testing_labels_final = np.array(y_valid).astype(int)

In [0]:
vocab_size = 5000
embedding_dim = 16
trunc_type='post'
oov_tok = "<OOV>"

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(x_train)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(x_train)
final_x_train = pad_sequences(sequences,truncating=trunc_type)
max_length = len(final_x_train[0])
testing_sequences = tokenizer.texts_to_sequences(x_valid)
final_x_valid = pad_sequences(testing_sequences,maxlen = max_length)


In [0]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    # tf.keras.layers.Flatten(),
    # tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True)),
    # tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
    # tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(8)),
    # tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [21]:
model.fit(final_x_train, training_labels_final, epochs=15, shuffle=True, validation_data=(final_x_valid, testing_labels_final),verbose = 2)

Epoch 1/15
13/13 - 1s - loss: 0.6898 - accuracy: 0.5652 - val_loss: 0.6843 - val_accuracy: 0.6031
Epoch 2/15
13/13 - 0s - loss: 0.6809 - accuracy: 0.5652 - val_loss: 0.6781 - val_accuracy: 0.6031
Epoch 3/15
13/13 - 0s - loss: 0.6694 - accuracy: 0.5652 - val_loss: 0.6699 - val_accuracy: 0.6031
Epoch 4/15
13/13 - 0s - loss: 0.6490 - accuracy: 0.5652 - val_loss: 0.6590 - val_accuracy: 0.6031
Epoch 5/15
13/13 - 0s - loss: 0.6105 - accuracy: 0.5831 - val_loss: 0.6465 - val_accuracy: 0.6031
Epoch 6/15
13/13 - 0s - loss: 0.5415 - accuracy: 0.7545 - val_loss: 0.6297 - val_accuracy: 0.6186
Epoch 7/15
13/13 - 0s - loss: 0.4223 - accuracy: 0.8798 - val_loss: 0.6332 - val_accuracy: 0.6546
Epoch 8/15
13/13 - 0s - loss: 0.2835 - accuracy: 0.9488 - val_loss: 0.6278 - val_accuracy: 0.7036
Epoch 9/15
13/13 - 0s - loss: 0.1869 - accuracy: 0.9744 - val_loss: 0.6677 - val_accuracy: 0.7139
Epoch 10/15
13/13 - 0s - loss: 0.1231 - accuracy: 0.9821 - val_loss: 0.7632 - val_accuracy: 0.7113
Epoch 11/15
13/13 -